# Prophet Forecasting Accuracy by Horizon and Window
This notebook compares forecasting performance across different time windows and prediction horizons.

- **Windows**: Historical starting points (from 90 to 30 minutes ago)
- **Horizons**: Forecast lengths (1, 2, 5, 10, 15 minutes)

For each combination, a Prophet model is trained and evaluated using MAE, RMSE, R², MAPE, and Accuracy (%).

In [ ]:
!pip install prophet openpyxl

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import os
import joblib

SAVE_DIR = "prophet_range_horizon_output"
os.makedirs(SAVE_DIR, exist_ok=True)


In [ ]:
df = pd.read_excel("usask.sec.min_short_v2.xlsx")
df.columns = ['minute', 'requests']
df['minute'] = pd.to_datetime(df['minute'], unit='m', origin='unix')
df.rename(columns={'minute': 'ds', 'requests': 'y'}, inplace=True)


In [ ]:
def evaluate_predictions(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    r2 = r2_score(y_true, y_pred)
    mape = mean_absolute_percentage_error(y_true, y_pred)
    accuracy = max(0, 100 * (1 - mape))
    return mae, rmse, r2, mape, accuracy


In [ ]:
windows = [-90, -75, -60, -45, -30]  # Historical points (minutes ago)
horizons = [1, 2, 5, 10, 15]         # Forecast lengths

results = []

for win in windows:
    for horizon in horizons:
        label = f"Window {abs(win)}min | Horizon {horizon}min"
        hist = df[:win]
        test = df[win:win + horizon]

        model = Prophet(
            growth='linear',
            yearly_seasonality=True,
            weekly_seasonality=True,
            daily_seasonality=True,
            changepoint_prior_scale=0.05,
            seasonality_prior_scale=10.0,
            holidays_prior_scale=10.0,
            n_changepoints=25
        )
        model.add_seasonality(name='hourly', period=1, fourier_order=3)
        model.fit(hist)

        future = model.make_future_dataframe(periods=horizon, freq='min')
        forecast = model.predict(future)

        # Evaluate
        merged = forecast[['ds', 'yhat']].merge(test, on='ds')
        mae, rmse, r2, mape, acc = evaluate_predictions(merged['y'], merged['yhat'])
        results.append((abs(win), horizon, mae, rmse, r2, mape, acc))

        # Plot
        plt.figure(figsize=(10, 3))
        plt.plot(merged['ds'], merged['y'], label='Actual', marker='o')
        plt.plot(merged['ds'], merged['yhat'], label='Forecast', marker='x')
        plt.title(f'{label}')
        plt.xlabel("Time"); plt.ylabel("Requests")
        plt.legend(); plt.grid(True)
        plt.savefig(f"{SAVE_DIR}/forecast_{abs(win)}min_{horizon}min.png")
        plt.show()


In [ ]:
result_df = pd.DataFrame(results, columns=['Window (min ago)', 'Horizon (min)', 'MAE', 'RMSE', 'R2', 'MAPE', 'Accuracy (%)'])
result_df.to_csv(f"{SAVE_DIR}/window_horizon_forecast_summary.csv", index=False)
result_df
